In [4]:
# default_exp dtm

In [5]:
from dynamic_topic_modeling.sklearn_lda import *

In [6]:
affirmative = make_df("affirmative.csv")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LIJIAX~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.954 seconds.
Prefix dict has been built succesfully.


In [7]:
affirmative.head()

,Group,Students,Content,text
0,第1组,正三,慕课将分布于世界各地的最优质的教育资源聚集到一起，让任何有学习愿望的人能够低成本的，通常是免...,慕课 将 分布 于 世界各地 的 最 优质 的 教育资源 聚集 到 一起 ， 让 任何 有 ...
1,第1组,正二,"在慕课发展过程中的现阶段，中国最大的慕课平台icourse163的用户人数突破100万,与其...",在 慕课 发展 过程 中 的 现阶段 ， 中国 最大 的 慕课 平台 icourse163 ...
2,第1组,正一,"研究发现,在慕课融入的课堂学习中,学习者情感体验丰富,知识技能以及元认知能力得到提升,思想观...","研究 发现 , 在 慕课 融入 的 课堂 学习 中 , 学习者 情感 体验 丰富 , 知识 ..."
3,第1组,正三,慕课在保证教育质量的同时，降低提供教育的成本，给社会带来的憧憬。任何人任何时候再任何地方，都...,慕课 在 保证 教育 质量 的 同时 ， 降低 提供 教育 的 成本 ， 给 社会 带来 的...
4,第1组,正一,对方反一辩友也说是可能出现的欢快气氛，传统课堂集体聆听教师单方面赐予的知识，这难道不是一种容...,对方 反一 辩友 也 说 是 可能 出现 的 欢快 气氛 ， 传统 课堂 集体 聆听 教师 ...


In [51]:
series_slices = affirmative['Group'] \
    .value_counts() \
    .reindex(affirmative['Group'].unique().tolist()) \
    .tolist()
# regex https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html
# reindex https://blog.csdn.net/songyunli1111/article/details/78953841

In [52]:
series_slices

[67, 135, 50, 53, 50, 47, 53, 68, 63, 31]

In [41]:
stopwords = get_custom_stopwords("stopwords.txt", encoding='utf-8') # HIT停用词词典
max_df = 0.9 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 5 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
n_features = 1000 # 最大提取特征数量
n_top_words = 20 # 显示主题下关键词的时候，显示多少个
col_content = "text" # 说明其中的文本信息所在列名称

In [42]:
raw_documents = affirmative['text'].tolist()

In [43]:
# 参考 https://blog.csdn.net/kwame211/article/details/78963517
import jieba
docs = [[word for word in jieba.cut(document, cut_all=True)] for document in raw_documents]

In [44]:
# 参考 https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
from gensim.corpora import Dictionary
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 5 documents, or more than 90% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.9)

```python
TypeError: doc2bow expects an array of unicode tokens on input, not a single string
```

- [x] list 化 https://blog.csdn.net/cg_amaz1ng/article/details/79567583

In [45]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [46]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1060
Number of documents: 617


In [55]:
from gensim.models.wrappers import DtmModel

# Set training parameters.
num_topics = 8
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

# 参考 https://radimrehurek.com/gensim/models/wrappers/dtmmodel.html
# dtm-win64.exe

model = DtmModel('dtm-win64.exe', corpus=corpus, id2word=id2word, num_topics = num_topics,
                 time_slices=series_slices)

In [56]:
# 参考 https://github.com/le-hoang-nhan/dynamic-topic-modeling
print(model.show_topic(topicid=1, time=0, topn=10))

[(0.054934152899280886, '学习'), (0.046114835910420926, '课'), (0.04495083634524851, '慕'), (0.0318348783258449, '课程'), (0.01555991198548604, '和'), (0.011949879053584214, '是'), (0.01155140324098257, '方式'), (0.011004917227570849, '在'), (0.010536307542231893, '学习者'), (0.009003648555199277, '可以')]


In [58]:
#Topic Evolution
num_topics = 8
for topic_no in range(num_topics):
    print("\nTopic", str(topic_no))
    for time in range(len(series_slices)):
        print("Time slice", str(time))
        print(model.show_topic(topic_no, time, topn=10))


Topic 0
Time slice 0
[(0.04018052022066546, '课'), (0.036249815551255456, '慕'), (0.02448548586657437, '是'), (0.023714387154559223, '发展'), (0.02367722760054125, '教育'), (0.017694317247958412, '我们'), (0.015624579186324783, '传统'), (0.015025505230014904, '在'), (0.012114008172785072, '了'), (0.011049820951041131, '反')]
Time slice 1
[(0.039142497272909985, '课'), (0.03579706784786207, '慕'), (0.024401378796314526, '是'), (0.023846475526857383, '教育'), (0.022735096814248044, '发展'), (0.017555946228496894, '我们'), (0.01571500622082442, '传统'), (0.014949393473384368, '在'), (0.012299193140748553, '了'), (0.010744284499093485, '反')]
Time slice 2
[(0.038475424316991955, '课'), (0.035365687723283065, '慕'), (0.024247191229078716, '是'), (0.023408284563598707, '教育'), (0.021401225289811407, '发展'), (0.01754042821642195, '我们'), (0.015669111450072614, '传统'), (0.014988245923534156, '在'), (0.01251012095645997, '了'), (0.010650328460717643, '反')]
Time slice 3
[(0.03748845503205829, '课'), (0.03517131753550811, '慕'), (0.0

In [59]:
#Distance between documents: compare the documents across different time-frames and see how similar they are topic-wise
#considering document 0
doc = 0
print("doc = 0, model.gamma_[doc]",model.gamma_[doc])

doc = 0, model.gamma_[doc] [1.88394876e-04 7.01846241e-01 1.88394876e-04 1.88394876e-04
 1.88595226e-01 1.88394876e-04 1.88394876e-04 1.08616559e-01]


In [60]:
#The distance between documents based on their topic distribution: lower, more related

from gensim.matutils import hellinger
# considering document 4 and 5
doc1 = 4
doc2 = 5
hellinger(model.gamma_[doc1], model.gamma_[doc2])

0.8113757952202133

In [61]:
import pyLDAvis

doc_topic, topic_term, doc_lengths, term_frequency, vocab = model.dtm_vis(time=0, corpus=corpus)
vis_wrapper = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_wrapper)

D:\install\miniconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [62]:
import pickle as pkl
with open("model/dtm.pkl", 'wb') as fp:
    pkl.dump(model, fp)

In [64]:
with open("model/dtm.pkl", 'rb') as fp:
    model0 = pkl.load(fp)
    print(model0.__class__)

<class 'gensim.models.wrappers.dtmmodel.DtmModel'>
